#### Leitura dos arquivos sobre o clima

Arquivos baixados do site [inmet](https://portal.inmet.gov.br/dadoshistoricos) (arquivos de 2022)

In [43]:
# Criar timestamp usando as colunas 'Data' e 'Hora UTC'
def create_timestamp(row):
    return dt.datetime.strptime(f"{row['Data']} {row['Hora UTC']}", "%Y/%m/%d %H%M UTC")

df_all_climate["timestamp"] = df_all_climate.apply(create_timestamp, axis=1)
df_all_climate = df_all_climate.drop(["Data", "Hora UTC"], axis=1)

In [44]:
df_all_climate = df_all_climate.rename_axis("id")
df_all_climate.to_csv("./data/fato_dados_meteorologicos.csv", sep=";")
df_all_climate.head(10)

,precipTotalHorario,ventoVeloHoraria,tempAr,radGlobal,umidRelAr,id_regiao,timestamp
id,,,,,,,
0,0.0,0.8,22.757692,17.638622,55.538462,0,2022-03-14 15:00:00
1,0.0,1.0,22.757692,0.000000,55.538462,0,2022-03-14 16:00:00
2,0.0,1.0,22.757692,0.000000,55.538462,0,2022-03-14 17:00:00
3,0.0,1.9,22.757692,0.000000,55.538462,0,2022-03-14 18:00:00
4,4.8,6.5,22.757692,0.000000,55.538462,0,2022-03-14 19:00:00
...,...,...,...,...,...,...,...
3437918,0.0,2.0,25.100000,275.600000,54.000000,566,2022-12-31 19:00:00
3437919,0.0,1.6,22.900000,11.600000,62.000000,566,2022-12-31 20:00:00
3437920,0.0,1.7,20.500000,1195.355160,76.000000,566,2022-12-31 21:00:00


In [7]:
df_grouped_climate = df_all_climate.set_index('timestamp')
df_grouped_climate = df_grouped_climate.groupby("timestamp").mean()
df_grouped_climate

,precipTotalHorario,ventoVeloHoraria,tempAr,radGlobal,umidRelAr,id_regiao
timestamp,,,,,,
2022-01-01 00:00:00,0.474844,1.607250,22.596342,1192.919960,81.318823,322.746914
2022-01-01 01:00:00,0.306536,1.606084,22.205653,1172.373779,83.259537,318.950156
2022-01-01 02:00:00,0.345876,1.565165,21.821098,1172.787725,84.494296,318.386997
2022-01-01 03:00:00,0.230854,1.532500,21.616310,1189.554431,85.013952,317.879154
2022-01-01 04:00:00,0.169650,1.534942,21.320228,1192.054759,86.043409,318.220183
...,...,...,...,...,...,...
2022-12-31 19:00:00,0.233127,2.292636,28.312385,1699.724795,58.892438,296.524838
2022-12-31 20:00:00,0.270761,2.170969,27.553467,1122.119650,62.001490,298.424508
2022-12-31 21:00:00,0.225741,1.874457,26.447883,568.365166,66.769132,298.385965
